In [1]:
import os, itertools, pickle
import numpy as np
import pandas as pd

from sklearn import feature_selection

RAW_PATH = "/data2/zhoujb/project/cowpea_project/rawData/"
SNP_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/snpDir/"
CLUSTER_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/SNPMMSeqCluster/"
RES_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/Result/"
ML_RAW_PATH = "/data2/zhoujb/project/cowpea_project/basedXPXLR/ML/rawData/"

def rmLowVarAndHighCor(data, feature_prefix='feature_', var_threshold=0.01, cor_threshods=0.8):
    
    data_feat = data.filter(regex='^{}'.format(feature_prefix), axis=1)
    
    # remove quasi-constant features
    # 0.01 indicates 99% of observations approximately
    var_sel = feature_selection.VarianceThreshold(threshold=var_threshold)
    var_sel.fit(data_feat)
    
    feat_keep = data_feat.columns[var_sel.get_support()]
    data_feat = data_feat[feat_keep]
    
    # drop highly correlated features
    corr_mat = data_feat.corr().abs()
    upper = corr_mat.where(np.triu(np.ones(corr_mat.shape), k=1).astype(np.bool_))
    to_drop = [column for column in upper.columns if any(upper[column] > cor_threshods)]
    
    feat_keep = [column for column in data_feat.columns if column not in to_drop]
    return feat_keep

In [4]:
# 表型数据
raw_phenos_1 = pd.read_excel(os.path.join(RAW_PATH, "Phenotypes for GWAS-upload public database.xlsx"), index_col="Accessions")
raw_phenos_2 = pd.read_excel(os.path.join(RAW_PATH, "BMK21067-AK808-data-2.xlsx"), index_col="BMK-ID")
raw_phenos_2 = raw_phenos_2[['HZ Pod Cellulose (mg/g)', 'GZ Pod Cellulose (mg/g)', 'HZ Seed Cellulose (mg/g)', 'GZ Seed Cellulose (mg/g)']].copy()
raw_phenos = raw_phenos_1.merge(raw_phenos_2, how='left', left_index=True, right_index=True)

raw_phenos = raw_phenos.drop(columns="ID")
raw_phenos = raw_phenos.replace({"-":np.nan})
raw_phenos = raw_phenos.dropna(axis=0, how="all")
raw_phenos = raw_phenos.rename(columns={'HZ-Pod length':'HZ-PL', 'GZ-Pod length':'GZ-PL', 
                                        'HZ Pod Sugar content (mg/g)':'HZ-PSugar','GZ Pod Sugar content  (mg/g)':'GZ-PSugar', 
                                        'HZ Pod Starch content(mg/g)':'HZ-PStarch','GZ Pod Strach Content (mg/g)':'GZ-PStarch', 
                                        'HZ Pod Protein content(g/Kg)':'HZ-PProtein','GZ Pod protein  (g/Kg)':'GZ-PProtein', 
                                        'HZ Seed Sugar (mg/g)':'HZ-SSuger','GZ Seed Sugar (mg/g)':'GZ-SSuger', 
                                        'HZ Seed Starch (mg/g)':'HZ-SStarch','GZ Seed Starch (mg/g)':'GZ-SStarch', 
                                        'HZ Seed Protein (g/kg)':'HZ-SProtein','GZ Seed Protein (g/kg)':'GZ-SProtein', 
                                        'Pod shattering':'PS', 'HZ Pod Cellulose (mg/g)':"HZ-PC", 'GZ Pod Cellulose (mg/g)':"GZ-PC",
                                        'HZ Seed Cellulose (mg/g)':"HZ-SC", 'GZ Seed Cellulose (mg/g)':"GZ-SC"})

target_df = raw_phenos[["GZ-PSugar"]].dropna()
keep_col = [x for x in raw_phenos.columns if (not x.startswith("HZ")) and (not x.endswith("PSugar"))]
pheno_data = raw_phenos[keep_col]
pheno_data = pheno_data.rename(columns={x:x.split("-")[-1] for x in pheno_data.columns})
pheno_data = pheno_data.rename(columns={x:"fea_{}".format(x) for x in pheno_data.columns})

# 基因组分数
genotype_hap_score = pd.read_table(os.path.join(RES_PATH, "genotype_hap_score.txt"), index_col=0)
genotype_hap_score = genotype_hap_score.rename(columns={x:"fea_{}".format(x) for x in genotype_hap_score.columns})

# 代谢组数据
deg_sample_val = pd.read_table(os.path.join(RAW_PATH, "metabolome_sample_value.txt"), index_col=0)
deg_sample_val = deg_sample_val.rename(columns={x:"fea_{}".format(x) for x in deg_sample_val.columns})

inte_sample = list(set(target_df.index).intersection(genotype_hap_score.index).intersection(deg_sample_val.index))

raw_data = pd.concat([genotype_hap_score.loc[inte_sample], deg_sample_val.loc[inte_sample], pheno_data.loc[inte_sample]], axis=1)
#raw_data = pd.concat([genotype_hap_score.loc[inte_sample], deg_sample_val.loc[inte_sample]], axis=1)
feat_keep = rmLowVarAndHighCor(raw_data, feature_prefix='fea_', var_threshold=0.01, cor_threshods=0.8)
raw_data = raw_data[feat_keep].copy()
raw_data = pd.concat([raw_data, target_df.loc[inte_sample]], axis=1)

# Write raw data
raw_data.to_csv(os.path.join(ML_RAW_PATH, "raw_data_PSugar.txt"), sep="\t")

In [5]:
raw_data

,fea_Chr01_51400001_51530000,fea_Chr01_4550001_4690000,fea_Chr01_44170001_44300000,fea_Chr01_5960001_6170000,fea_Chr01_34570001_34760000,fea_Chr01_2330001_2630000,fea_Chr01_43270001_43430000,fea_Chr02_680001_1040000,fea_Chr01_49970001_50400000,fea_Chr01_46910001_47010000,...,fea_PStarch,fea_PProtein,fea_SSuger,fea_SStarch,fea_SProtein,fea_PS,fea_TSW,fea_PC,fea_SC,GZ-PSugar
D371,0.820745,0.468997,0.652068,0.390666,0.800345,0.013596,0.760746,0.609217,2.032302,0.096657,...,15.052083,22.890777,77.401615,242.509587,261.700,0.0,168.1163,44.391760,693.405473,15.23340
D441,0.012978,0.468997,0.652068,0.390666,0.800345,0.859828,0.760746,0.609217,0.028427,0.618527,...,19.536400,26.627741,81.638440,228.822799,247.440,1.0,104.0560,69.056493,658.669487,11.47185
D460,0.820745,0.206476,0.652068,0.390666,0.008652,0.151889,0.008652,0.609217,2.032302,0.096657,...,19.117350,21.587130,88.434393,200.116096,240.760,0.0,107.0068,57.672160,735.864335,16.00560
D674,0.012978,0.468997,0.082849,0.390666,0.800345,0.859828,0.096657,0.609217,0.028427,0.008652,...,21.000667,21.398974,84.652778,218.969288,253.500,0.0,146.7826,66.573360,696.049199,10.41105
D619,0.820745,0.206476,0.652068,0.004944,0.008652,0.013596,0.760746,0.609217,2.032302,0.618527,...,14.804025,25.231339,87.403667,205.261324,284.620,0.0,141.0000,29.905493,594.444167,13.58370
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D393,0.820745,0.468997,0.652068,0.390666,0.800345,0.859828,0.760746,0.609217,2.032302,0.618527,...,20.225183,19.428992,90.732657,208.276349,270.020,0.0,145.8373,NaN,645.461048,6.38820
D593,0.012978,0.468997,0.652068,0.390666,0.800345,0.013596,0.760746,0.609217,2.032302,0.618527,...,15.201400,24.370084,75.781056,326.544036,291.680,0.0,114.9592,54.958960,818.253935,15.49275
D529,0.820745,0.206476,0.652068,0.004944,0.800345,0.859828,0.760746,0.609217,2.032302,0.618527,...,11.709317,23.590690,75.367521,233.492654,285.395,0.0,NaN,33.158160,621.861312,14.32860
D645,0.820745,0.468997,0.652068,0.390666,0.800345,0.859828,0.760746,0.609217,2.032302,0.618527,...,11.410683,23.077281,87.851037,289.219447,278.205,0.0,173.9435,50.928827,595.316145,10.88100
